In [ ]:
### Extract surface areas and correlate with p-factor

In [1]:
from pathlib import Path
import pandas as pd

# Check both shared directories
shared_dirs = [
    Path('/home/jovyan/shared'),
    Path('/home/jovyan/shared-readwrite')
]

print("Checking shared directories for participant data...")

for shared_dir in shared_dirs:
    print(f"\n{'='*50}")
    print(f"CHECKING: {shared_dir}")
    print(f"{'='*50}")
    
    if shared_dir.exists():
        print("Contents:")
        for item in shared_dir.iterdir():
            if item.is_file():
                print(f"  📄 {item.name}")
                # Check if it's a TSV file
                if item.suffix == '.tsv':
                    try:
                        # Read first few rows to check structure
                        df_sample = pd.read_csv(item, sep='\t', nrows=3)
                        print(f"    Columns: {list(df_sample.columns)}")
                        
                        # Check for participant data indicators
                        has_participant = any('participant' in col.lower() or 'subject' in col.lower() 
                                            for col in df_sample.columns)
                        has_p_factor = any('p_factor' in col.lower() or 'pfactor' in col.lower() 
                                         for col in df_sample.columns)
                        
                        if has_participant:
                            print("    ✓ Has participant/subject column")
                        if has_p_factor:
                            print("    ✓ Has p_factor column")
                        
                        if has_participant and has_p_factor:
                            print("    🎯 THIS IS YOUR PARTICIPANT DATA FILE!")
                            
                            # Load the full file
                            print(f"    Loading full file...")
                            all_data = pd.read_csv(item, sep='\t')
                            print(f"    Shape: {all_data.shape}")
                            print(f"    Training participants: {len(all_data[~pd.isna(all_data['p_factor'])])}")
                            print(f"    Test participants: {len(all_data[pd.isna(all_data['p_factor'])])}")
                            
                            # Show sample
                            print("    Sample data:")
                            print(all_data.head(3))
                            
                    except Exception as e:
                        print(f"    Error reading TSV: {e}")
                        
            elif item.is_dir():
                print(f"  📁 {item.name}/")
                # Check subdirectories briefly
                try:
                    subitems = list(item.iterdir())[:3]
                    for subitem in subitems:
                        print(f"     - {subitem.name}")
                    if len(list(item.iterdir())) > 3:
                        print(f"     ... and more")
                except:
                    pass
    else:
        print("Directory doesn't exist")

# Also check for any CSV files
print(f"\n{'='*50}")
print("CHECKING FOR CSV FILES")
print(f"{'='*50}")

for shared_dir in shared_dirs:
    if shared_dir.exists():
        csv_files = list(shared_dir.glob('*.csv'))
        if csv_files:
            print(f"CSV files in {shared_dir}:")
            for csv_file in csv_files:
                print(f"  {csv_file.name}")

# Template for loading once found
print(f"\n{'='*50}")
print("NEXT STEPS")
print(f"{'='*50}")
print("Once you find the participant data file, load it with:")
print("all_data = pd.read_csv('/path/to/participant/file.tsv', sep='\\t')")
print("Then run the surface area analysis.")

Checking shared directories for participant data...

CHECKING: /home/jovyan/shared
Contents:
  📁 data/
     - RBC
     - abide2
     - .ipynb_checkpoints
     ... and more
  📁 docs/
     - .git
     - Personal-Release-Form.pdf
     - git-setup.html
     ... and more

CHECKING: /home/jovyan/shared-readwrite
Directory doesn't exist

CHECKING FOR CSV FILES

NEXT STEPS
Once you find the participant data file, load it with:
all_data = pd.read_csv('/path/to/participant/file.tsv', sep='\t')
Then run the surface area analysis.


In [2]:
from pathlib import Path
import pandas as pd

# Check the RBC data directory
rbc_dir = Path('/home/jovyan/shared/data/RBC')

print(f"Checking RBC data directory: {rbc_dir}")

if rbc_dir.exists():
    print("RBC directory contents:")
    
    def explore_directory(directory, max_depth=2, current_depth=0):
        """Recursively explore directory looking for TSV files"""
        if current_depth > max_depth:
            return
        
        indent = "  " * current_depth
        
        for item in directory.iterdir():
            if item.is_file():
                print(f"{indent}📄 {item.name}")
                
                # Check TSV files for participant data
                if item.suffix.lower() in ['.tsv', '.csv']:
                    try:
                        df_sample = pd.read_csv(item, sep='\t' if item.suffix == '.tsv' else ',', nrows=3)
                        print(f"{indent}   Columns: {list(df_sample.columns)}")
                        
                        # Look for participant indicators
                        has_participant = any('participant' in col.lower() or 'subject' in col.lower() 
                                            for col in df_sample.columns)
                        has_p_factor = any('p_factor' in col.lower() or 'pfactor' in col.lower() 
                                         for col in df_sample.columns)
                        
                        if has_participant:
                            print(f"{indent}   ✓ Has participant/subject data")
                        if has_p_factor:
                            print(f"{indent}   ✓ Has p_factor data")
                        
                        if has_participant and has_p_factor:
                            print(f"{indent}   🎯 PARTICIPANT DATA FOUND!")
                            
                            # Load full file
                            all_data = pd.read_csv(item, sep='\t' if item.suffix == '.tsv' else ',')
                            print(f"{indent}   Full shape: {all_data.shape}")
                            print(f"{indent}   Training: {len(all_data[~pd.isna(all_data['p_factor'])])}")
                            print(f"{indent}   Test: {len(all_data[pd.isna(all_data['p_factor'])])}")
                            
                            return item  # Return the path to the file
                            
                    except Exception as e:
                        print(f"{indent}   Error reading: {e}")
                        
            elif item.is_dir():
                print(f"{indent}📁 {item.name}/")
                result = explore_directory(item, max_depth, current_depth + 1)
                if result:  # If we found the participant data, return it
                    return result
    
    # Explore the RBC directory
    participant_file = explore_directory(rbc_dir)
    
    if participant_file:
        print(f"\n{'='*60}")
        print("PARTICIPANT DATA FILE FOUND!")
        print(f"{'='*60}")
        print(f"File: {participant_file}")
        print("\nLoad it with:")
        print(f"all_data = pd.read_csv('{participant_file}', sep='\\t')")
        
    else:
        print(f"\n{'='*60}")
        print("NO PARTICIPANT DATA FOUND IN RBC DIRECTORY")
        print(f"{'='*60}")
        print("The participant data might be:")
        print("1. In a different subdirectory")
        print("2. Named differently")
        print("3. Need to be generated from other files")
        
        # Look for any files with 'train', 'test', 'participant', 'subject' in name
        print("\nLooking for files with relevant names...")
        for item in rbc_dir.rglob('*'):
            if item.is_file():
                name_lower = item.name.lower()
                if any(keyword in name_lower for keyword in ['train', 'test', 'participant', 'subject', 'pnc']):
                    print(f"  Found: {item}")

else:
    print("RBC directory doesn't exist")
    
    # Check what's actually in the data directory
    data_dir = Path('/home/jovyan/shared/data')
    if data_dir.exists():
        print(f"\nContents of {data_dir}:")
        for item in data_dir.iterdir():
            print(f"  {item.name}")

Checking RBC data directory: /home/jovyan/shared/data/RBC
RBC directory contents:
📁 PNC_FreeSurfer/
  📁 .git/
    📄 description
    📁 hooks/
    📁 info/
    📄 config
    📁 objects/
    📄 HEAD
    📁 refs/
    📄 packed-refs
    📁 logs/
    📄 index
  📁 .datalad/
    📄 .gitattributes
    📄 config
  📄 .gitattributes
  📄 .gitignore
  📄 .gitmodules
  📄 CHANGELOG.md
  📄 README.md
  📁 code/
    📄 .gitattributes
    📄 README.md
    📄 extract_freesurfer.sh
    📄 merge_outputs.sh
    📄 participant_job.sh
    📄 qsub_array.sh
    📄 subject_ids.txt
  📁 freesurfer/
    📁 sub-1000393599/
    📁 sub-1000881804/
    📁 sub-1001970838/
    📁 sub-100527940/
    📁 sub-1006151876/
    📁 sub-1007995238/
    📁 sub-1011497669/
    📁 sub-1012530688/
    📁 sub-1017092387/
    📁 sub-1020132225/
    📁 sub-1023701836/
    📁 sub-1025031408/
    📁 sub-1027389114/
    📁 sub-1030193285/
    📁 sub-103652366/
    📁 sub-1037644062/
    📁 sub-1039797417/
    📁 sub-1039936647/
    📁 sub-1043634177/
    📁 sub-1044438549/
    📁 

In [ ]:
### Look for the training file

In [ ]:
import pandas as pd
from pathlib import Path

# Look for training participants file
rbc_dir = Path('/home/jovyan/shared/data/RBC')

print("Looking for training participants file...")
for file in rbc_dir.rglob('*train*participants*.tsv'):
    print(f"Found potential training file: {file}")
    
    try:
        df = pd.read_csv(file, sep='\t')
        print(f"  Shape: {df.shape}")
        print(f"  Columns: {list(df.columns)}")
        
        # Check for p_factor
        p_factor_cols = [col for col in df.columns if 'p_factor' in col.lower()]
        if p_factor_cols:
            p_col = p_factor_cols[0]
            non_nan = len(df[~pd.isna(df[p_col])])
            print(f"  P-factor column: {p_col}")
            print(f"  Non-NaN p_factor: {non_nan}")
            print(f"  NaN p_factor: {len(df) - non_nan}")
    except Exception as e:
        print(f"  Error reading: {e}")

# Also check for any file with "train" in the name
print("\nLooking for any files with 'train' in name...")
for file in rbc_dir.rglob('*train*.tsv'):
    print(f"Found: {file}")

# Check root RBC directory specifically
print(f"\nChecking root RBC directory: {rbc_dir}")
for file in rbc_dir.glob('*.tsv'):
    print(f"Found TSV: {file}")
    if 'train' in file.name.lower() or 'participants' in file.name.lower():
        try:
            df = pd.read_csv(file, sep='\t', nrows=3)
            print(f"  Quick peek at {file.name}: {df.shape[1]} columns")
        except:
            pass

# Let's also check what we know about the total split
print(f"\nData split verification:")
test_file = '/home/jovyan/shared/data/RBC/test_participants.tsv'
test_data = pd.read_csv(test_file, sep='\t')
print(f"Test participants: {len(test_data)}")

bids_file = '/home/jovyan/shared/data/RBC/PNC_BIDS/study-PNC_desc-participants.tsv'
bids_data = pd.read_csv(bids_file, sep='\t')
print(f"BIDS total participants: {len(bids_data)}")
print(f"Expected training size: {len(bids_data) - len(test_data)} = {1601 - 534}")

Looking for training participants file...
Found potential training file: /home/jovyan/shared/data/RBC/train_participants.tsv
  Shape: (1067, 19)
  Columns: ['participant_id', 'study', 'study_site', 'session_id', 'wave', 'age', 'sex', 'race', 'ethnicity', 'bmi', 'handedness', 'participant_education', 'parent_1_education', 'parent_2_education', 'p_factor', 'internalizing_mcelroy_harmonized_all_samples', 'externalizing_mcelroy_harmonized_all_samples', 'attention_mcelroy_harmonized_all_samples', 'cubids_acquisition_group']
  P-factor column: p_factor
  Non-NaN p_factor: 1067
  NaN p_factor: 0


In [ ]:
# Check age and sex data in training set
import pandas as pd
import numpy as np

# Load training data
train_file = '/home/jovyan/shared/data/RBC/train_participants.tsv'
all_data = pd.read_csv(train_file, sep='\t')

print("DEMOGRAPHIC DATA CHECK")
print("=" * 50)

print(f"Total participants: {len(all_data)}")
print(f"Columns: {list(all_data.columns)}")

# Check age data
print(f"\nAGE DATA:")
print(f"  Missing age: {all_data['age'].isna().sum()}")
print(f"  Age range: {all_data['age'].min():.1f} - {all_data['age'].max():.1f}")
print(f"  Mean age: {all_data['age'].mean():.1f}")
print(f"  Age distribution:")
print(all_data['age'].describe())

# Check sex data  
print(f"\nSEX DATA:")
print(f"  Missing sex: {all_data['sex'].isna().sum()}")
print(f"  Sex distribution:")
print(all_data['sex'].value_counts())

# Check p_factor
print(f"\nP_FACTOR DATA:")
print(f"  Missing p_factor: {all_data['p_factor'].isna().sum()}")
print(f"  P_factor range: {all_data['p_factor'].min():.3f} - {all_data['p_factor'].max():.3f}")
print(f"  P_factor distribution:")
print(all_data['p_factor'].describe())

# Check complete cases
complete_cases = all_data.dropna(subset=['age', 'sex', 'p_factor'])
print(f"\nCOMPLETE CASES (age + sex + p_factor):")
print(f"  Complete cases: {len(complete_cases)}")
print(f"  Dropped: {len(all_data) - len(complete_cases)}")

# Sample of data
print(f"\nSAMPLE DATA:")
print(all_data[['participant_id', 'age', 'sex', 'p_factor']].head(10))

# Check if the partial correlation killed all the signal
print(f"\nBEFORE vs AFTER ADJUSTMENT CHECK:")
print("If partial correlation reduced all correlations below 0.1,")
print("it might be over-controlling or there's an issue with the method.")
print("\nLet's also check the raw correlations without adjustment...")

# Quick correlation check between age/sex and p_factor
from scipy.stats import spearmanr, pearsonr

print(f"\nCORRELATIONS WITH P_FACTOR:")
if len(complete_cases) > 0:
    # Age vs p_factor
    age_corr, age_p = spearmanr(complete_cases['age'], complete_cases['p_factor'])
    print(f"  Age vs p_factor: r={age_corr:.3f}, p={age_p:.3f}")
    
    # Sex vs p_factor (convert sex to numeric first)
    complete_cases_copy = complete_cases.copy()
    complete_cases_copy['sex_numeric'] = (complete_cases_copy['sex'] == 'Male').astype(int)
    sex_corr, sex_p = spearmanr(complete_cases_copy['sex_numeric'], complete_cases_copy['p_factor'])
    print(f"  Sex vs p_factor: r={sex_corr:.3f}, p={sex_p:.3f}")
    
    print(f"\nIf age/sex have strong correlations with p_factor,")
    print(f"partial correlation might be removing meaningful variance.")

In [5]:
# Extract ALL FreeSurfer features and compute partial correlations
import pandas as pd
import numpy as np
from rbclib import RBCPath
from scipy.stats import spearmanr
from ipywidgets import IntProgress
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

def partial_spearman_correlation(x, y, covariates):
    """Compute partial spearman correlation controlling for covariates"""
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        for i, cov in enumerate(covariates.T):
            data[f'cov_{i}'] = cov
        
        data = data.dropna()
        if len(data) < 10:
            return np.nan, np.nan, len(data)
        
        cov_cols = [col for col in data.columns if col.startswith('cov_')]
        X_cov = data[cov_cols].values
        X_cov = np.column_stack([np.ones(len(X_cov)), X_cov])
        
        try:
            beta_x = np.linalg.lstsq(X_cov, data['x'], rcond=None)[0]
            beta_y = np.linalg.lstsq(X_cov, data['y'], rcond=None)[0]
            
            residual_x = data['x'] - X_cov @ beta_x
            residual_y = data['y'] - X_cov @ beta_y
            
            r, p = spearmanr(residual_x, residual_y)
            return r, p, len(data)
            
        except np.linalg.LinAlgError:
            return np.nan, np.nan, len(data)
    except:
        return np.nan, np.nan, 0

# Load training data
train_file = '/home/jovyan/shared/data/RBC/train_participants.tsv'
all_data = pd.read_csv(train_file, sep='\t')

all_data_clean = all_data.dropna(subset=['age', 'sex', 'p_factor']).copy()
all_data_clean['sex_numeric'] = (all_data_clean['sex'] == 'Male').astype(int)

print(f"Loaded {len(all_data_clean)} participants with complete data")

def load_all_brain_features(participant_id):
    """Load ALL brain measurements for a participant"""
    try:
        sub_path = RBCPath(f'rbc://PNC_FreeSurfer/freesurfer/sub-{participant_id}')
        stats_filepath = sub_path / f'sub-{participant_id}_regionsurfacestats.tsv'
        
        with stats_filepath.open('r') as f:
            data = pd.read_csv(f, sep='\t')
        return data
    except:
        return None

# Explore what features are available
print("Exploring ALL available brain features...")
sample_subject = all_data_clean['participant_id'].iloc[0]
sample_data = load_all_brain_features(sample_subject)

if sample_data is None:
    print("❌ Could not load sample data")
else:
    print(f"Sample data shape: {sample_data.shape}")
    print(f"ALL columns: {list(sample_data.columns)}")
    
    # Categorize features by type
    all_columns = sample_data.columns.tolist()
    
    # Remove non-numeric identifier columns
    exclude_cols = ['participant_id', 'subject_id', 'sub', 'hemisphere', 'hemi']
    brain_features = [col for col in all_columns 
                     if not any(excl in col.lower() for excl in exclude_cols)]
    
    print(f"\nBrain feature columns ({len(brain_features)}):")
    
    # Categorize features
    surface_features = [col for col in brain_features if 'area' in col.lower() or 'surf' in col.lower()]
    volume_features = [col for col in brain_features if 'vol' in col.lower()]
    thickness_features = [col for col in brain_features if 'thick' in col.lower()]
    curv_features = [col for col in brain_features if 'curv' in col.lower() or 'curve' in col.lower()]
    other_features = [col for col in brain_features 
                     if col not in surface_features + volume_features + thickness_features + curv_features]
    
    print(f"  Surface area features: {len(surface_features)} - {surface_features}")
    print(f"  Volume features: {len(volume_features)} - {volume_features}")  
    print(f"  Thickness features: {len(thickness_features)} - {thickness_features}")
    print(f"  Curvature features: {len(curv_features)} - {curv_features}")
    print(f"  Other features: {len(other_features)} - {other_features}")
    
    # Use ALL brain features
    print(f"\nUsing ALL {len(brain_features)} brain features")
    
    # Extract features for all participants
    print(f"Extracting features for {len(all_data_clean)} participants...")
    
    feature_dict = {
        'participant_id': [],
        'p_factor': [],
        'age': [],
        'sex_numeric': []
    }
    for col in brain_features:
        feature_dict[col] = []
    
    # Progress bar
    prog = IntProgress(min=0, max=len(all_data_clean))
    display(prog)
    
    missing_count = 0
    for idx, row in all_data_clean.iterrows():
        participant_id = row['participant_id']
        
        feature_dict['participant_id'].append(participant_id)
        feature_dict['p_factor'].append(row['p_factor'])
        feature_dict['age'].append(row['age'])
        feature_dict['sex_numeric'].append(row['sex_numeric'])
        
        brain_data = load_all_brain_features(participant_id)
        
        if brain_data is not None and len(brain_data) > 0:
            for col in brain_features:
                if col in brain_data.columns:
                    value = brain_data[col].iloc[0]
                    feature_dict[col].append(value)
                else:
                    feature_dict[col].append(np.nan)
        else:
            missing_count += 1
            for col in brain_features:
                feature_dict[col].append(np.nan)
        
        prog.value += 1
    
    print(f"\nFeature extraction complete! Missing data for {missing_count} participants")
    
    # Convert to dataframe
    features_df = pd.DataFrame(feature_dict)
    
    # Compute partial correlations for ALL brain features
    print("Computing partial spearman correlations for ALL brain features...")
    
    correlations = []
    
    for column in brain_features:
        complete_data = features_df[['p_factor', column, 'age', 'sex_numeric']].dropna()
        
        if len(complete_data) < 10:
            correlations.append({
                'feature': column,
                'feature_type': 'unknown',
                'n_subjects': len(complete_data),
                'spearman_r': np.nan,
                'spearman_p': np.nan
            })
            continue
        
        # Categorize feature type
        if column in surface_features:
            feat_type = 'surface_area'
        elif column in volume_features:
            feat_type = 'volume'
        elif column in thickness_features:
            feat_type = 'thickness'
        elif column in curv_features:
            feat_type = 'curvature'
        else:
            feat_type = 'other'
        
        # Prepare data for partial correlation
        y = complete_data['p_factor'].values
        x = complete_data[column].values
        covariates = complete_data[['age', 'sex_numeric']].values
        
        # Compute partial correlation
        r, p, n = partial_spearman_correlation(x, y, covariates)
        
        correlations.append({
            'feature': column,
            'feature_type': feat_type,
            'n_subjects': n,
            'spearman_r': r,
            'spearman_p': p
        })
    
    # Results
    correlation_df = pd.DataFrame(correlations)
    correlation_df['abs_spearman_r'] = np.abs(correlation_df['spearman_r'])
    correlation_df = correlation_df.sort_values('abs_spearman_r', ascending=False)
    
    print(f"\n{'='*90}")
    print("ALL BRAIN FEATURES - PARTIAL CORRELATIONS (controlling for age and sex)")
    print(f"{'='*90}")
    
    print(f"Total brain features analyzed: {len(correlation_df)}")
    valid_corrs = correlation_df[~correlation_df['spearman_r'].isna()]
    print(f"Valid correlations computed: {len(valid_corrs)}")
    
    # Save ALL results to CSV
    output_file = 'all_brain_features_correlations_partial.csv'
    correlation_df.to_csv(output_file, index=False)
    print(f"\n✓ ALL brain feature correlations saved to: {output_file}")
    
    # Show top 20 results
    print(f"\nTOP 20 BRAIN FEATURE CORRELATIONS:")
    print(f"{'Rank':<5} {'Feature':<35} {'Type':<15} {'r':<8} {'p-value':<10} {'n':<6}")
    print("-" * 85)
    
    for i, (_, row) in enumerate(correlation_df.head(20).iterrows(), 1):
        if pd.isna(row['spearman_r']):
            print(f"{i:4d}  {row['feature']:<35} {row['feature_type']:<15} {'NaN':<8} {'NaN':<10} {row['n_subjects']:5.0f}")
        else:
            direction = "↑" if row['spearman_r'] > 0 else "↓"
            print(f"{i:4d}  {direction} {row['feature']:<33} {row['feature_type']:<15} {row['spearman_r']:6.3f}   {row['spearman_p']:8.3f}   {row['n_subjects']:5.0f}")
    
    # Summary by feature type
    print(f"\nSUMMARY BY FEATURE TYPE:")
    for feat_type in ['surface_area', 'volume', 'thickness', 'curvature', 'other']:
        type_data = valid_corrs[valid_corrs['feature_type'] == feat_type]
        if len(type_data) > 0:
            print(f"  {feat_type.upper()}:")
            print(f"    Count: {len(type_data)}")
            print(f"    Mean |r|: {type_data['abs_spearman_r'].mean():.3f}")
            print(f"    Max |r|: {type_data['abs_spearman_r'].max():.3f}")
            print(f"    Significant (p<0.05): {len(type_data[type_data['spearman_p'] < 0.05])}")

Loaded 1067 participants with complete data
Exploring ALL available brain features...
Sample data shape: (13740, 27)
ALL columns: ['subject_id', 'session_id', 'atlas', 'hemisphere', 'StructName', 'NumVert', 'SurfArea', 'GrayVol', 'ThickAvg', 'ThickStd', 'MeanCurv', 'GausCurv', 'FoldInd', 'CurvInd', 'Index', 'SegId', 'Mean_wgpct', 'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct', 'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi', 'Range_piallgi']

Brain feature columns (25):
  Surface area features: 1 - ['SurfArea']
  Volume features: 1 - ['GrayVol']
  Thickness features: 2 - ['ThickAvg', 'ThickStd']
  Curvature features: 3 - ['MeanCurv', 'GausCurv', 'CurvInd']
  Other features: 18 - ['session_id', 'atlas', 'StructName', 'NumVert', 'FoldInd', 'Index', 'SegId', 'Mean_wgpct', 'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct', 'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi', 'Range_piallgi']

Using ALL 25 brain features
Ext

IntProgress(value=0, max=1067)


Feature extraction complete! Missing data for 7 participants
Computing partial spearman correlations for ALL brain features...

ALL BRAIN FEATURES - PARTIAL CORRELATIONS (controlling for age and sex)
Total brain features analyzed: 25
Valid correlations computed: 22

✓ ALL brain feature correlations saved to: all_brain_features_correlations_partial.csv

TOP 20 BRAIN FEATURE CORRELATIONS:
Rank  Feature                             Type            r        p-value    n     
-------------------------------------------------------------------------------------
   1  ↓ Min_piallgi                       other           -0.112      0.000    1047
   2  ↓ GrayVol                           volume          -0.109      0.000    1060
   3  ↓ SurfArea                          surface_area    -0.104      0.001    1060
   4  ↓ NumVert                           other           -0.102      0.001    1060
   5  ↓ Mean_piallgi                      other           -0.100      0.001    1047
   6  ↓ Max_piallg

In [4]:
# explore what atlases and structures are available in the brain data
import pandas as pd
import numpy as np
from rbclib import RBCPath

# load one participant's brain data to see what's available
train_file = '/home/jovyan/shared/data/RBC/train_participants.tsv'
all_data = pd.read_csv(train_file, sep='\t')

# get first participant
first_participant = all_data.iloc[0]['participant_id']
print(f"exploring brain data structure using participant: {first_participant}")

try:
    sub_path = RBCPath(f'rbc://PNC_FreeSurfer/freesurfer/sub-{first_participant}')
    stats_filepath = sub_path / f'sub-{first_participant}_regionsurfacestats.tsv'
    
    with stats_filepath.open('r') as f:
        sample_data = pd.read_csv(f, sep='\t')
    
    print(f"total brain regions/structures: {len(sample_data)}")
    print(f"columns available: {list(sample_data.columns)}")
    
    # check what atlases are available
    print(f"\navailable atlases:")
    atlas_counts = sample_data['atlas'].value_counts()
    for atlas, count in atlas_counts.items():
        print(f"  {atlas}: {count} regions")
    
    # show sample of each atlas
    print(f"\nsample structures by atlas:")
    for atlas in sample_data['atlas'].unique():
        atlas_data = sample_data[sample_data['atlas'] == atlas]
        print(f"\n{atlas.upper()} atlas:")
        print(f"  columns: {list(atlas_data.columns)}")
        
        # show first few structures
        structures = atlas_data[['StructName', 'hemisphere']].drop_duplicates().head(10)
        print(f"  sample structures:")
        for _, row in structures.iterrows():
            hemi = f"({row['hemisphere']})" if pd.notna(row['hemisphere']) else "(no hemi)"
            print(f"    {row['StructName']} {hemi}")
        
        if len(atlas_data) > 10:
            print(f"    ... and {len(atlas_data) - 10} more")
        
        # check what measures are available for this atlas
        numeric_cols = atlas_data.select_dtypes(include=[np.number]).columns
        print(f"  numeric measures: {list(numeric_cols)}")
    
    # specifically look for subcortical structures
    print(f"\nlooking for subcortical structures...")
    subcortical_keywords = ['hippocampus', 'amygdala', 'caudate', 'putamen', 'pallidum', 'thalamus', 'accumbens']
    
    for keyword in subcortical_keywords:
        matches = sample_data[sample_data['StructName'].str.contains(keyword, case=False, na=False)]
        if len(matches) > 0:
            print(f"  found {keyword}: {len(matches)} entries")
            for _, match in matches.iterrows():
                hemi = f"({match['hemisphere']})" if pd.notna(match['hemisphere']) else ""
                print(f"    {match['StructName']} {hemi} [{match['atlas']}]")
    
    # check if there are volume measures for subcortical
    print(f"\nchecking volume measures...")
    vol_cols = [col for col in sample_data.columns if 'vol' in col.lower()]
    print(f"  volume columns: {vol_cols}")
    
    if vol_cols:
        for vol_col in vol_cols:
            non_na = sample_data[vol_col].notna().sum()
            print(f"    {vol_col}: {non_na}/{len(sample_data)} non-null values")

except Exception as e:
    print(f"error loading brain data: {e}")
    print("this might help debug the issue with accessing brain files")

exploring brain data structure using participant: 1000393599
total brain regions/structures: 13740
columns available: ['subject_id', 'session_id', 'atlas', 'hemisphere', 'StructName', 'NumVert', 'SurfArea', 'GrayVol', 'ThickAvg', 'ThickStd', 'MeanCurv', 'GausCurv', 'FoldInd', 'CurvInd', 'Index', 'SegId', 'Mean_wgpct', 'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct', 'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi', 'Range_piallgi']

available atlases:
  Schaefer2018_1000Parcels_7Networks_order: 1002 regions
  Schaefer2018_1000Parcels_17Networks_order: 1002 regions
  Schaefer2018_900Parcels_17Networks_order: 902 regions
  Schaefer2018_900Parcels_7Networks_order: 902 regions
  Schaefer2018_800Parcels_7Networks_order: 802 regions
  Schaefer2018_800Parcels_17Networks_order: 802 regions
  Schaefer2018_700Parcels_7Networks_order: 702 regions
  Schaefer2018_700Parcels_17Networks_order: 702 regions
  Slab: 693 regions
  Schaefer2018_600Parcels_7Networks_order

In [5]:
# AAL atlas volume analysis - comprehensive cortical + subcortical analysis
import pandas as pd
import numpy as np
from rbclib import RBCPath
from scipy.stats import spearmanr
from ipywidgets import IntProgress
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

def partial_spearman_correlation(x, y, covariates):
    """compute partial spearman correlation controlling for covariates"""
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        for i, cov in enumerate(covariates.T):
            data[f'cov_{i}'] = cov
        
        data = data.dropna()
        if len(data) < 10:
            return np.nan, np.nan, len(data)
        
        cov_cols = [col for col in data.columns if col.startswith('cov_')]
        X_cov = data[cov_cols].values
        X_cov = np.column_stack([np.ones(len(X_cov)), X_cov])
        
        try:
            beta_x = np.linalg.lstsq(X_cov, data['x'], rcond=None)[0]
            beta_y = np.linalg.lstsq(X_cov, data['y'], rcond=None)[0]
            
            residual_x = data['x'] - X_cov @ beta_x
            residual_y = data['y'] - X_cov @ beta_y
            
            r, p = spearmanr(residual_x, residual_y)
            return r, p, len(data)
            
        except np.linalg.LinAlgError:
            return np.nan, np.nan, len(data)
    except:
        return np.nan, np.nan, 0

# load training data
train_file = '/home/jovyan/shared/data/RBC/train_participants.tsv'
all_data = pd.read_csv(train_file, sep='\t')

all_data_clean = all_data.dropna(subset=['age', 'sex', 'p_factor']).copy()
all_data_clean['sex_numeric'] = (all_data_clean['sex'] == 'Male').astype(int)

print(f"loaded {len(all_data_clean)} participants with complete data")

def load_regional_brain_data(participant_id):
    """load brain data for a participant"""
    try:
        sub_path = RBCPath(f'rbc://PNC_FreeSurfer/freesurfer/sub-{participant_id}')
        stats_filepath = sub_path / f'sub-{participant_id}_regionsurfacestats.tsv'
        
        with stats_filepath.open('r') as f:
            data = pd.read_csv(f, sep='\t')
        data['participant_id'] = participant_id
        return data
    except:
        return None

# load all brain data for all participants at once
print("loading brain data for all participants (this may take a few minutes)...")

all_brain_data_list = []
failed_participants = []

# progress bar for loading
load_prog = IntProgress(min=0, max=len(all_data_clean))
display(load_prog)

for i, (_, participant_row) in enumerate(all_data_clean.iterrows()):
    participant_id = participant_row['participant_id']
    
    if i % 100 == 0:
        print(f"  loading participant {i+1}/{len(all_data_clean)}...")
    
    brain_data = load_regional_brain_data(participant_id)
    
    if brain_data is not None:
        # add participant info to brain data
        brain_data['p_factor'] = participant_row['p_factor']
        brain_data['age'] = participant_row['age']
        brain_data['sex_numeric'] = participant_row['sex_numeric']
        all_brain_data_list.append(brain_data)
    else:
        failed_participants.append(participant_id)
    
    load_prog.value += 1

print(f"\nloaded brain data for {len(all_brain_data_list)} participants")
print(f"failed to load: {len(failed_participants)} participants")

if len(all_brain_data_list) > 0:
    # combine all brain data into one big dataframe
    print("combining all brain data...")
    all_brain_data = pd.concat(all_brain_data_list, ignore_index=True)
    
    print(f"combined brain data shape: {all_brain_data.shape}")
    print(f"unique participants: {all_brain_data['participant_id'].nunique()}")
    
    # filter to AAL atlas only (includes both cortical and subcortical)
    aal_data = all_brain_data[all_brain_data['atlas'] == 'AAL'].copy()
    print(f"AAL data shape: {aal_data.shape}")
    
    # get unique AAL regions
    unique_regions = aal_data[['StructName', 'hemisphere']].drop_duplicates()
    print(f"unique AAL regions: {len(unique_regions)}")
    
    # categorize structures as cortical vs subcortical for analysis
    subcortical_keywords = ['hippocampus', 'amygdala', 'caudate', 'putamen', 'pallidum', 'thalamus', 'accumbens']
    
    def is_subcortical(struct_name):
        return any(keyword in struct_name.lower() for keyword in subcortical_keywords)
    
    # now compute correlations for each AAL region
    print("computing volume correlations for each AAL region...")
    
    volume_correlations = []
    
    # progress bar for correlations
    corr_prog = IntProgress(min=0, max=len(unique_regions))
    display(corr_prog)
    
    for i, (_, region_row) in enumerate(unique_regions.iterrows()):
        struct_name = region_row['StructName']
        hemisphere = region_row['hemisphere']
        
        # get data for this specific region
        region_data = aal_data[
            (aal_data['StructName'] == struct_name) & 
            (aal_data['hemisphere'] == hemisphere)
        ].copy()
        
        if len(region_data) > 0:
            # compute partial correlation for volume
            complete_data = region_data[['p_factor', 'GrayVol', 'age', 'sex_numeric']].dropna()
            
            if len(complete_data) < 10:
                volume_correlations.append({
                    'structure': struct_name,
                    'hemisphere': hemisphere,
                    'region_id': f"{struct_name}_{hemisphere}",
                    'region_type': 'subcortical' if is_subcortical(struct_name) else 'cortical',
                    'n_subjects': len(complete_data),
                    'spearman_r': np.nan,
                    'spearman_p': np.nan
                })
            else:
                # compute partial correlation
                y = complete_data['p_factor'].values
                x = complete_data['GrayVol'].values
                covariates = complete_data[['age', 'sex_numeric']].values
                
                r, p, n = partial_spearman_correlation(x, y, covariates)
                
                volume_correlations.append({
                    'structure': struct_name,
                    'hemisphere': hemisphere,
                    'region_id': f"{struct_name}_{hemisphere}",
                    'region_type': 'subcortical' if is_subcortical(struct_name) else 'cortical',
                    'n_subjects': n,
                    'spearman_r': r,
                    'spearman_p': p
                })
        
        corr_prog.value += 1
    
    # convert results to dataframe
    results_df = pd.DataFrame(volume_correlations)
    results_df['abs_spearman_r'] = np.abs(results_df['spearman_r'])
    results_df = results_df.sort_values('abs_spearman_r', ascending=False)
    
    print(f"\n{'='*90}")
    print("AAL VOLUME CORRELATIONS (partial correlations - age/sex adjusted)")
    print(f"{'='*90}")
    
    valid_results = results_df[~results_df['spearman_r'].isna()]
    print(f"valid correlations: {len(valid_results)}/{len(results_df)}")
    
    # separate subcortical and cortical results
    subcortical_results = valid_results[valid_results['region_type'] == 'subcortical']
    cortical_results = valid_results[valid_results['region_type'] == 'cortical']
    
    print(f"subcortical regions: {len(subcortical_results)}")
    print(f"cortical regions: {len(cortical_results)}")
    
    # save results
    output_file = 'aal_volume_correlations.csv'
    results_df.to_csv(output_file, index=False)
    print(f"\n✓ results saved to: {output_file}")
    
    # show subcortical results first (most interesting for psychiatry)
    if len(subcortical_results) > 0:
        print(f"\n🧠 SUBCORTICAL VOLUME CORRELATIONS:")
        print(f"{'rank':<5} {'structure':<25} {'hemi':<6} {'r':<8} {'p-value':<10} {'n':<6}")
        print("-" * 75)
        
        subcortical_sorted = subcortical_results.sort_values('abs_spearman_r', ascending=False)
        for i, (_, row) in enumerate(subcortical_sorted.iterrows(), 1):
            if pd.isna(row['spearman_r']):
                print(f"{i:4d}  {row['structure']:<25} {row['hemisphere']:<6} {'nan':<8} {'nan':<10} {row['n_subjects']:5.0f}")
            else:
                direction = "↑" if row['spearman_r'] > 0 else "↓"
                sig_marker = "*" if row['spearman_p'] < 0.05 else " "
                print(f"{i:4d}  {direction} {row['structure']:<23} {row['hemisphere']:<6} {row['spearman_r']:6.3f}   {row['spearman_p']:8.3f}{sig_marker}  {row['n_subjects']:5.0f}")
    
    # show top cortical results
    print(f"\n🧠 TOP 20 CORTICAL VOLUME CORRELATIONS:")
    print(f"{'rank':<5} {'structure':<25} {'hemi':<6} {'r':<8} {'p-value':<10} {'n':<6}")
    print("-" * 75)
    
    cortical_sorted = cortical_results.sort_values('abs_spearman_r', ascending=False).head(20)
    for i, (_, row) in enumerate(cortical_sorted.iterrows(), 1):
        if pd.isna(row['spearman_r']):
            print(f"{i:4d}  {row['structure']:<25} {row['hemisphere']:<6} {'nan':<8} {'nan':<10} {row['n_subjects']:5.0f}")
        else:
            direction = "↑" if row['spearman_r'] > 0 else "↓"
            sig_marker = "*" if row['spearman_p'] < 0.05 else " "
            print(f"{i:4d}  {direction} {row['structure']:<23} {row['hemisphere']:<6} {row['spearman_r']:6.3f}   {row['spearman_p']:8.3f}{sig_marker}  {row['n_subjects']:5.0f}")
    
    # summary statistics
    if len(valid_results) > 0:
        significant_results = valid_results[valid_results['spearman_p'] < 0.05]
        strong_results = valid_results[valid_results['abs_spearman_r'] > 0.1]
        
        subcortical_sig = subcortical_results[subcortical_results['spearman_p'] < 0.05]
        cortical_sig = cortical_results[cortical_results['spearman_p'] < 0.05]
        
        print(f"\nsummary:")
        print(f"  total AAL regions: {len(valid_results)}")
        print(f"  mean |r|: {valid_results['abs_spearman_r'].mean():.3f}")
        print(f"  max |r|: {valid_results['abs_spearman_r'].max():.3f}")
        print(f"  significant (p < 0.05): {len(significant_results)}")
        print(f"  strong (|r| > 0.1): {len(strong_results)}")
        print(f"  subcortical significant: {len(subcortical_sig)}")
        print(f"  cortical significant: {len(cortical_sig)}")
        
        if len(significant_results) > 0:
            print(f"\nmost significant correlations:")
            top_sig = significant_results.sort_values('spearman_p').head(10)
            for _, row in top_sig.iterrows():
                direction = "↑" if row['spearman_r'] > 0 else "↓"
                region_type = "🧠" if row['region_type'] == 'subcortical' else "🧮"
                print(f"  {region_type} {direction} {row['structure']} ({row['hemisphere']}): r={row['spearman_r']:.3f}, p={row['spearman_p']:.3f}")
        
        print(f"\n* = significant at p < 0.05")
        print(f"🧠 = subcortical, 🧮 = cortical")
        print(f"all correlations are partial correlations controlling for age and sex")

else:
    print("❌ no brain data loaded successfully")

loaded 1067 participants with complete data
loading brain data for all participants (this may take a few minutes)...


IntProgress(value=0, max=1067)

  loading participant 1/1067...
  loading participant 101/1067...
  loading participant 201/1067...
  loading participant 301/1067...
  loading participant 401/1067...
  loading participant 501/1067...
  loading participant 601/1067...
  loading participant 701/1067...
  loading participant 801/1067...
  loading participant 901/1067...
  loading participant 1001/1067...

loaded brain data for 1060 participants
failed to load: 7 participants
combining all brain data...
combined brain data shape: (14561756, 31)
unique participants: 1060
AAL data shape: (105038, 31)
unique AAL regions: 100
computing volume correlations for each AAL region...


IntProgress(value=0)


AAL VOLUME CORRELATIONS (partial correlations - age/sex adjusted)
valid correlations: 100/100
subcortical regions: 9
cortical regions: 91

✓ results saved to: aal_volume_correlations.csv

🧠 SUBCORTICAL VOLUME CORRELATIONS:
rank  structure                 hemi   r        p-value    n     
---------------------------------------------------------------------------
   1  ↓ Hippocampus_R           rh     -0.074      0.015*   1060
   2  ↓ Thalamus_L              lh     -0.063      0.040*   1060
   3  ↑ Pallidum_R              rh      0.053      0.198     598
   4  ↓ Caudate_R               rh     -0.034      0.266    1060
   5  ↓ Thalamus_R              rh     -0.032      0.300    1060
   6  ↓ Amygdala_R              rh     -0.026      0.404    1060
   7  ↓ Hippocampus_L           lh     -0.015      0.627    1060
   8  ↑ Amygdala_L              lh      0.013      0.675    1060
   9  ↓ Caudate_L               lh     -0.009      0.823     609

🧠 TOP 20 CORTICAL VOLUME CORRELATIONS:
rank  str